In [9]:
import torch
import torch.nn as nn
import torch.functional as F
from torch.optim import Adam
import numpy as np
import lightning as light
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd

In [10]:
def read_in_tables(n=78):
    df_list = {}
    for i in range(1, n):

        table1_name = 'test_df' + str(i)
        table2_name = 'train_df' + str(i)

        df = pd.read_csv(f'../datasets/ml_{i}.csv', parse_dates=['date'])

        globals()[table1_name] = df[(df.date <= pd.to_datetime('2019-12-31')) & (df.date >= pd.to_datetime('2019-01-01'))]
        globals()[table2_name] = df[(df.date < pd.to_datetime('2019-01-01')) & (df.date > pd.to_datetime('2016-01-01'))]

        globals()[table1_name].fillna(0, inplace=True)
        globals()[table2_name].fillna(0, inplace=True)

        globals()[table1_name].set_index('date', inplace=True)
        globals()[table2_name].set_index('date', inplace=True)

        df_list[table1_name] = globals()[table1_name]
        df_list[table2_name] = globals()[table2_name]

    return df_list

tables = read_in_tables()

C:\Users\adamt\AppData\Local\Temp\ipykernel_5868\1737775152.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  globals()[table1_name].fillna(0, inplace=True)
C:\Users\adamt\AppData\Local\Temp\ipykernel_5868\1737775152.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  globals()[table2_name].fillna(0, inplace=True)
C:\Users\adamt\AppData\Local\Temp\ipykernel_5868\1737775152.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  global

In [20]:
class SimpleLSTM(light.LightningModule):
    def __init__(self, cols):
        super().__init__()
        mean = torch.tensor(0.0)
        std = torch.tensor(1.0)
        locals().update({f'{col}': {'wlr1' : nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True),
                                    'wlr2' : nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True),
                                    'blr1' : nn.Parameter(torch.tensor(0.), requires_grad=True),

                                    'wpr1' : nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True),
                                    'wpr2' : nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True),
                                    'bpr1' : nn.Parameter(torch.tensor(0.), requires_grad=True),

                                    'wp1' : nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True),
                                    'wp2' : nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True),
                                    'bp1' : nn.Parameter(torch.tensor(0.), requires_grad=True),

                                    'wo1' : nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True),
                                    'wo2' : nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True),
                                    'bo1' : nn.Parameter(torch.tensor(0.), requires_grad=True),
                                    } for col in cols})
    
    # input_value = ["col1", "col2", "col3"]
    def lstm_unit(self, input_values, input_names, long_memory, short_memory):
        
        length = len(input_names)

        long_remember_percent = 0
        potential_remember_percent = 0
        potential_memory = 0
        # This could be fucked
        for i in range(length):
            long_remember_percent += torch.sigmoid((short_memory * locals()[input_names[i]]['wlr1']) + 
                                                  (input_values[i] * locals()[input_names[i]]['wlr2']) +
                                                  locals()[input_names[i]]['blr1'])
        long_remember_percent /= length
        
        for i in range(length):
            potential_remember_percent += torch.sigmoid((short_memory * locals()[input_names[i]]['wpr1']) + 
                                                  (input_values[i] * locals()[input_names[i]]['wpr2']) +
                                                  locals()[input_names[i]]['bpr1'])
                
            potential_memory += torch.tanh((short_memory * locals()[input_names[i]]['wp1']) + 
                                           (input_values[i] * locals()[input_names[i]]['wp2']) +
                                           locals()[input_names[i]]['bp1'])
            
        potential_remember_percent /= length
        potential_memory /= length 
        
        updated_long_memory = ((long_memory * long_remember_percent) + 
                               (potential_remember_percent * potential_memory))
        for i in range(length):
            output_percent += torch.tanh((short_memory * locals()[input_names[i]]['wo1']) + 
                                        (input_values[i] * locals()[input_names[i]]['wo2']) +
                                        locals()[input_names[i]]['bo1'])
        output_percent /= length

        updated_short_memory = torch.tanh(updated_long_memory) * output_percent
        
        return([updated_long_memory, updated_short_memory])
    
    def forward(self, input):
        # forward function
        pass
    
    def configure_optimizers():
        return Adam(self.parameters())
    
    def training_steps():
        pass